In [1]:
# Installation des librairies tensorflow
!pip install tensorflow
# Et tensorflow_datasets pour récupérer la BDD
!pip install tensorflow_datasets

In [2]:
# Importation des librairies
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib 
import pandas as pd 
import numpy as np
tf.__version__

'2.7.0-dev20210902'

In [3]:
# Importation de la BDD
data = pd.read_csv("https://go.aws/314bBDq",encoding="utf-8")

In [4]:
# Preview de la BDD
data.head()

,user_id,review,stars,date_format,time_of_day,hour_of_day,day_of_week,review_format,review_lang,month_year,review_len,review_nb_words
0,efb62a167fee5cf3678b24427de8e31f,"Génial, fabuleux, exceptionnel ! J'aimerais qu...",5,2017-09-29 18:17:00,18:17,18,Ven,génial fabuleux exceptionnel j aimerais qu...,french,2017-09,115,19
1,e3be4f9c9e0b9572bfb2a5f88497bb14,NaN,2,2017-09-29 17:29:00,17:29,17,Ven,NaN,NaN,2017-09,0,0
2,1b8e5760162d867e9b9ca80f645bdc60,"Toujours aussi magic, féerique !",5,2017-09-29 16:46:00,16:46,16,Ven,toujours aussi magic féerique,french,2017-09,32,4
3,fa330e5891a1bb486c3e9bf95c098726,NaN,5,2017-09-29 15:52:00,15:52,15,Ven,NaN,NaN,2017-09,0,0
4,c1a693206aee1a2412d4bd9e45b80ec5,NaN,3,2017-09-29 15:29:00,15:29,15,Ven,NaN,NaN,2017-09,0,0


In [5]:
# On ne garde que les avis en français
dataset= data.loc[data["review_lang"]=="french"]

In [6]:
# Preview de la BDD en français
dataset.head()

,user_id,review,stars,date_format,time_of_day,hour_of_day,day_of_week,review_format,review_lang,month_year,review_len,review_nb_words
0,efb62a167fee5cf3678b24427de8e31f,"Génial, fabuleux, exceptionnel ! J'aimerais qu...",5,2017-09-29 18:17:00,18:17,18,Ven,génial fabuleux exceptionnel j aimerais qu...,french,2017-09,115,19
2,1b8e5760162d867e9b9ca80f645bdc60,"Toujours aussi magic, féerique !",5,2017-09-29 16:46:00,16:46,16,Ven,toujours aussi magic féerique,french,2017-09,32,4
11,726b1a3e2664e8b075129bcd643dbf56,En vacances en région parisienne nous nous som...,2,2017-09-29 00:37:00,00:37,0,Ven,en vacances en région parisienne nous nous som...,french,2017-09,172,25
12,8a71763fbb3da7436b957681b24cc404,Tropbeaufinalpleinlesyeuxoreil,5,2017-09-29 00:16:00,00:16,0,Ven,tropbeaufinalpleinlesyeuxoreil,french,2017-09,30,1
23,ce7abd7798ee036d667c0ad84b85daa7,L'univers Disney reste merveilleux. Toutefois ...,4,2017-09-28 20:24:00,20:24,20,Jeu,l univers disney reste merveilleux toutefois ...,french,2017-09,148,23


In [7]:
# On ne garde que les colonnes des avis et du nombre d'étoile par observation
columns = ["review_format","stars"]
datafinal= dataset[columns]

In [8]:
# Preview de la BDD cleané
datafinal.head()

,review_format,stars
0,génial fabuleux exceptionnel j aimerais qu...,5
2,toujours aussi magic féerique,5
11,en vacances en région parisienne nous nous som...,2
12,tropbeaufinalpleinlesyeuxoreil,5
23,l univers disney reste merveilleux toutefois ...,4


In [9]:
# On supprime les données non renseignées
datafinal.dropna()

,review_format,stars
0,génial fabuleux exceptionnel j aimerais qu...,5
2,toujours aussi magic féerique,5
11,en vacances en région parisienne nous nous som...,2
12,tropbeaufinalpleinlesyeuxoreil,5
23,l univers disney reste merveilleux toutefois ...,4
...,...,...
295057,toujours aussi magique même si à la fin du séj...,5
295549,séjour au top mes enfants les plus heureux ...,5
298475,magnifique un monde parfait lt,5
298832,oui j ai aimé car j adore disney et tout ce qu...,4


In [10]:
# Regardons le nombre de catégories pour le nombre d'étoiles
print(datafinal["stars"].unique())
# On en a 5

[5 2 4 3 1]


In [11]:
# Dimensions de la BDD
datafinal.shape

(8474, 2)

In [12]:
# On installe spacy
!pip install spacy
# Pour télécharger la pipeline de tokenization
!python -m spacy download fr_core_news_sm
import fr_core_news_sm
# On enregistre la pipeline dans la variable NLP 
nlp = fr_core_news_sm.load()

2021-09-27 16:40:22.260236: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-09-27 16:40:22.260402: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[+] Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [13]:
# On importe la séquence de mot que l'on  va enlever dans notre corpus de ligne
# STOP_WORDS
from spacy.lang.fr.stop_words import STOP_WORDS

In [14]:
# On lemmatize le texte
# Tout d'abord on rassemble les lignes de chaque observations pour en faire un paragraphe
datafinal["review_format_clean"] = datafinal["review_format"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
# Puis on remplace le caractère + entre chaque ligne par un espace et on supprime tous les espaces en trop
datafinal["review_format_clean"] = datafinal["review_format"].apply(lambda x: x.replace(" +"," ").lower().strip())
# On lemmatize les mots (on les traduit par leur racine commune) avec fr_core_news_sm en enlevant ceux compris dans liste de STOP_WORDS
datafinal["review_format_clean"] = datafinal["review_format"].apply(lambda x:" ".join([token.lemma_ for token in nlp(x) if token.lemma_ not in STOP_WORDS]))

<ipython-input-14-c5a16d1f8834>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datafinal["review_format_clean"] = datafinal["review_format"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
<ipython-input-14-c5a16d1f8834>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datafinal["review_format_clean"] = datafinal["review_format"].apply(lambda x: x.replace(" +"," ").lower().strip())
<ipython-input-14-c5a16d1f8834>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [45]:
# On visualise le résultat
datafinal

,review_format,stars,review_format_clean,tokenize_text
0,génial fabuleux exceptionnel j aimerais qu...,5,génial fabuleux exceptionnel j aimerai ...,"[91, 546, 465, 7, 1754, 407, 8, 5113, 1755]"
2,toujours aussi magic féerique,5,magic féerique,"[311, 65]"
11,en vacances en région parisienne nous nous som...,2,vacance région parisien décider visiter parc r...,"[379, 3044, 1756, 780, 528, 5, 2088, 3725, 118..."
12,tropbeaufinalpleinlesyeuxoreil,5,tropbeaufinalpleinlesyeuxoreil,[5114]
23,l univers disney reste merveilleux toutefois ...,4,l univers disney merveilleux toutefois regre...,"[4, 366, 8, 83, 1243, 644, 35, 55, 1895, 101, ..."
...,...,...,...,...
295057,toujours aussi magique même si à la fin du séj...,5,magique fin séjour rotule lol,"[10, 175, 31, 10169, 401]"
295549,séjour au top mes enfants les plus heureux ...,5,séjour top enfant heureux vouloir voir per...,"[31, 44, 16, 296, 94, 24, 47, 141, 31, 45, 565..."
298475,magnifique un monde parfait lt,5,magnifique monde parfaire lt,"[25, 29, 113, 151]"
298832,oui j ai aimé car j adore disney et tout ce qu...,4,oui j aimer j adore disney touche univers ...,"[178, 7, 186, 7, 76, 8, 2212, 366, 42, 16, 403..."


In [16]:
# On instancie la méthode  de tokenization dans une variable tokenize
tokenize= tf.keras.preprocessing.text.Tokenizer()

In [17]:
# Puis on applique la tokenization sur le text clean des avis
tokenize.fit_on_texts(datafinal["review_format_clean"])

In [18]:
#~Regardons de quoi à l'air la liste de token et le numéro associé à chaque mot
tokenize.word_index

{'d': 1,
 'c': 2,
 'attraction': 3,
 'l': 4,
 'parc': 5,
 'faire': 6,
 'j': 7,
 'disney': 8,
 'attente': 9,
 'magique': 10,
 'journée': 11,
 'n': 12,
 'heure': 13,
 'bien': 14,
 'trop': 15,
 'enfant': 16,
 'aller': 17,
 'fois': 18,
 'pouvoir': 19,
 'passer': 20,
 'super': 21,
 'an': 22,
 'petit': 23,
 'voir': 24,
 'magnifique': 25,
 'fermer': 26,
 'prix': 27,
 'magie': 28,
 'monde': 29,
 'jour': 30,
 'séjour': 31,
 'bon': 32,
 'beaucoup': 33,
 'spectacle': 34,
 'qu': 35,
 'personnel': 36,
 'temps': 37,
 'vraiment': 38,
 'disneyland': 39,
 'parade': 40,
 'dommage': 41,
 'grand': 42,
 'plein': 43,
 'top': 44,
 'venir': 45,
 'hôtel': 46,
 'personnage': 47,
 'oeil': 48,
 'fille': 49,
 'cher': 50,
 'rien': 51,
 's': 52,
 'moment': 53,
 'revenir': 54,
 'falloir': 55,
 'déçu': 56,
 'pann': 57,
 'famille': 58,
 'file': 59,
 'attendre': 60,
 'minute': 61,
 'prendre': 62,
 'payer': 63,
 'queue': 64,
 'féerique': 65,
 'mickey': 66,
 'beau': 67,
 'noël': 68,
 'retourner': 69,
 'endroit': 70,
 'sup

In [19]:
# On enregistre les phrases tokenizer dans une nouvelle colonne de notre dataframe
# Avec la méthode texts_to_sequences
datafinal["tokenize_text"]= tokenize.texts_to_sequences(datafinal["review_format_clean"])

<ipython-input-19-8ba99660fb83>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datafinal["tokenize_text"]= tokenize.texts_to_sequences(datafinal["review_format_clean"])


In [20]:
# Jetons un oeil à notre BDD finale
datafinal

,review_format,stars,review_format_clean,tokenize_text
0,génial fabuleux exceptionnel j aimerais qu...,5,génial fabuleux exceptionnel j aimerai ...,"[91, 546, 465, 7, 1754, 407, 8, 5113, 1755]"
2,toujours aussi magic féerique,5,magic féerique,"[311, 65]"
11,en vacances en région parisienne nous nous som...,2,vacance région parisien décider visiter parc r...,"[379, 3044, 1756, 780, 528, 5, 2088, 3725, 118..."
12,tropbeaufinalpleinlesyeuxoreil,5,tropbeaufinalpleinlesyeuxoreil,[5114]
23,l univers disney reste merveilleux toutefois ...,4,l univers disney merveilleux toutefois regre...,"[4, 366, 8, 83, 1243, 644, 35, 55, 1895, 101, ..."
...,...,...,...,...
295057,toujours aussi magique même si à la fin du séj...,5,magique fin séjour rotule lol,"[10, 175, 31, 10169, 401]"
295549,séjour au top mes enfants les plus heureux ...,5,séjour top enfant heureux vouloir voir per...,"[31, 44, 16, 296, 94, 24, 47, 141, 31, 45, 565..."
298475,magnifique un monde parfait lt,5,magnifique monde parfaire lt,"[25, 29, 113, 151]"
298832,oui j ai aimé car j adore disney et tout ce qu...,4,oui j aimer j adore disney touche univers ...,"[178, 7, 186, 7, 76, 8, 2212, 366, 42, 16, 403..."


In [21]:
# On met tous les lignes à la même taille de token
# Avec la méthode pad_sequences
# Sinon le modèle ne peut pas s'entraîner
# On enregistre la liste de phrases tokenizé et mis à la même longueur dans la variable pad_review
pad_review = tf.keras.preprocessing.sequence.pad_sequences(datafinal["tokenize_text"],padding="post")

In [46]:
pad_review

array([[  91,  546,  465, ...,    0,    0,    0],
       [ 311,   65,    0, ...,    0,    0,    0],
       [ 379, 3044, 1756, ...,    0,    0,    0],
       ...,
       [  25,   29,  113, ...,    0,    0,    0],
       [ 178,    7,  186, ...,    0,    0,    0],
       [  17,    8,  386, ...,    0,    0,    0]])

In [22]:
# On enregistre les 2 colonnes dont on a besoin dans une nouvelle variable
# C'est ce dont on va se servir pour entraîner notre modèle puis le tester
encode_data =tf.data.Dataset.from_tensor_slices((pad_review,datafinal["stars"].values-1))

In [49]:
# On choisit la taille de notre échantillon d'entraînement
# Ici on prend 80 % de la taille de notre dataset
size = int(len(datafinal["stars"])*0.8)
all_encoded_data = encode_data.shuffle(len(datafinal["stars"]))

In [50]:
# On enregistre dans la variable train_data l'échantillon d'entraînement
train_data = all_encoded_data.take(size)
# On groupe les données par 16 afin d'optimiser et de stabiliser l'entraînement du modèle
train_data = train_data.batch(16)

# On enregistre dans la variable test_data l'échantillon qui nous permet de tester le modèle
test_data= all_encoded_data.skip(size)
# On groupe également les données par 16 
test_data = test_data.batch(16)

In [51]:
# On regarde que les deux commandes ont bien marché sur nos échantillons
for review,stars in test_data.take(1):
    print(review, stars)

tf.Tensor(
[[223  41  97 ...   0   0   0]
 [ 21 473  17 ...   0   0   0]
 [ 28  37   1 ...   0   0   0]
 ...
 [ 28 214  33 ...   0   0   0]
 [ 32  11   3 ...   0   0   0]
 [ 10   0   0 ...   0   0   0]], shape=(16, 248), dtype=int32) tf.Tensor([4 3 4 2 4 4 3 4 4 3 4 4 4 1 3 4], shape=(16,), dtype=int64)


In [61]:
len(tokenize.word_index)

10169

In [52]:
# On enregistre la taille du vocabulaire de notre dataset dans la variable vocab_size
vocab_size = len(tokenize.word_index)

# On créé le modèle de réseaux de neurones qui va nous permettre de créer notre modèle prédictif
model = tf.keras.Sequential([
    # La première couche du réseau est celle qui va recevoir l'ensemble du dictionnaire de mot
    # Ici on fixe donc la taille de neurones à la taille du dictionnaire +1 (car le compte commence à 0 en python)
    tf.keras.layers.Embedding(vocab_size+1,64,input_shape=[review.shape[1],]),
    # On ajoute une couche de neurones de convolutions qui va synthétiser l'information sortant de la première couche
    # L'output ne sera que sur 1  seule dimensions
    tf.keras.layers.Conv1D(16,3, activation='relu'),
    # La couche Flatten va transformer les outputs de la couche précèdente (dimension 16*3)
    # En un vecteur d'input pour les prochaines couches (dimension 48*1)
    tf.keras.layers.Flatten(),
    # On utilise alors des couches de réseaux avec 64 neurones
    tf.keras.layers.Dense(64,activation="relu"),
    # Puis 32
    tf.keras.layers.Dense(32,activation="relu"),
    # Puis 16
    tf.keras.layers.Dense(16,activation="relu"),
    # Puis 8
    tf.keras.layers.Dense(8, activation="relu"),
    # Et enfin, la dernière couche comportera autant de neurones (et donc d'output)
    # Que notre variable target 
    # Qui ici possède 5 modalités (de 1 à 5 étoiles par avis)
    tf.keras.layers.Dense(5,activation="softmax")
])

In [53]:
# Implémentation du schéma d'apprentissage du modèle

## On fixe le taux d'apprentissage à 0.1 % ce qui est la norme
learning_rate = 0.001

## On enregistre dans la variable lr_schedule
## L'ensemble des éléments de notre schéma d'apprentissage
lr_schedule= tf.keras.optimizers.schedules.ExponentialDecay(
                # On met le taux d'apprentissage qu'on a enregistré
                learning_rate,
                # Nombre d'étape avant la diminution du taux d'apprentissage du modèle
                decay_steps= 10000,
                # De combien décroit le taux d'apprentissage? ici de 96%
                decay_rate= 0.96,
                # Cet argument staircase= True veut juste dire qu'on va suivre
                # Un schéma de décroissance du taux d'apprentissage en escalier
                # Qui dépendra du taux d'apprentissage et du nombre de pas prévu avant la fin 
                staircase=True                
)

In [54]:
# On fixe l'optimiseur qu'on choisit pour notre modèle
optimizer= tf.keras.optimizers.Adam(
    # On fixe le schéma d'apprentissage de notre modèle
    
    learning_rate=lr_schedule
)

In [55]:
## Compilation du modèle
model.compile(optimizer=optimizer, # On définit un optimiseur
             loss= tf.keras.losses.SparseCategoricalCrossentropy(), # On définit la mesure de la fonction de perte
             metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]) # On définit la métrique de précision du modèle

In [56]:
## Entraînement du modèle
# Sur notre échantillon d'entraînement
# Le nombre d'epochs correspond au nombre de fois que le modèle va parcourir notre dataset 
history = model.fit(train_data,epochs=40,validation_data=test_data)

Epoch 1/40
424/424 [==============================] - 5s 10ms/step - loss: 1.0158 - sparse_categorical_accuracy: 0.6138 - val_loss: 0.8250 - val_sparse_categorical_accuracy: 0.6667
Epoch 2/40
424/424 [==============================] - 4s 10ms/step - loss: 0.8157 - sparse_categorical_accuracy: 0.6690 - val_loss: 0.7038 - val_sparse_categorical_accuracy: 0.7215
Epoch 3/40
424/424 [==============================] - 4s 9ms/step - loss: 0.7150 - sparse_categorical_accuracy: 0.7056 - val_loss: 0.6121 - val_sparse_categorical_accuracy: 0.7475
Epoch 4/40
424/424 [==============================] - 4s 9ms/step - loss: 0.6180 - sparse_categorical_accuracy: 0.7578 - val_loss: 0.5280 - val_sparse_categorical_accuracy: 0.7988
Epoch 5/40
424/424 [==============================] - 4s 9ms/step - loss: 0.5323 - sparse_categorical_accuracy: 0.7933 - val_loss: 0.4312 - val_sparse_categorical_accuracy: 0.8383
Epoch 6/40
424/424 [==============================] - 4s 9ms/step - loss: 0.4380 - sparse_categori

In [68]:
# On regarde si toutes nos classes sont représentés de manière homogène
datafinal["stars"].value_counts()
# On voit que ce n'est pas le cas ici
# On va donc modifier légèrement notre modèle
# Pour qu'on ait des poids quasi-uniformes pour chaque catégorie

5    4881
4    1538
3    1010
1     558
2     487
Name: stars, dtype: int64

In [117]:
# On rajoute un paramètre supplémentaire
# Les weight vont paramètrer le poids de chaque catégorie de notre variable target
# Dans l'apprentissage de notre modèle
# Ici, les rang 4 et 5 sont sur-représentés dans notre dataset
# Donc en diminue le poids
# On fait l'inverse pour les rang 1 et 2 qui sont sous-représentés
weight_dic= {
    0:0,
    1:2,
    2:2,
    3:1,
    4:0.7,
    5:0.30
}

In [118]:
# On ré-entraîne notre modèle, cette fois ci avec les poids en paramètres
history = model.fit(train_data,epochs=40,validation_data=test_data,class_weight=weight_dic)

Epoch 1/40
424/424 [==============================] - 4s 10ms/step - loss: 0.0034 - sparse_categorical_accuracy: 0.9987 - val_loss: 0.0081 - val_sparse_categorical_accuracy: 0.9959
Epoch 2/40
424/424 [==============================] - 4s 10ms/step - loss: 0.0088 - sparse_categorical_accuracy: 0.9960 - val_loss: 0.0078 - val_sparse_categorical_accuracy: 0.9965
Epoch 3/40
424/424 [==============================] - 4s 9ms/step - loss: 0.0082 - sparse_categorical_accuracy: 0.9979 - val_loss: 0.0019 - val_sparse_categorical_accuracy: 0.9994
Epoch 4/40
424/424 [==============================] - 4s 9ms/step - loss: 0.0054 - sparse_categorical_accuracy: 0.9976 - val_loss: 0.0028 - val_sparse_categorical_accuracy: 0.9988
Epoch 5/40
424/424 [==============================] - 4s 9ms/step - loss: 0.0041 - sparse_categorical_accuracy: 0.9982 - val_loss: 0.0071 - val_sparse_categorical_accuracy: 0.9971
Epoch 6/40
424/424 [==============================] - 4s 9ms/step - loss: 0.0043 - sparse_categori

In [115]:
## Essai sur un batch
for i in range(15):
    for example, sh in test_data.take(1):
        print(i)
        print("Catégorie prédite : {}".format(np.argmax(model.predict(example)[i])))
        print("Véritable catégorie : {}".format(sh[i]))
        print("Probabilité de la prédiction : {} %".format(np.max(model.predict(example)[i] * 100).astype("float")))

0
Catégorie prédite : 3
Véritable catégorie : 3
Probabilité de la prédiction : 99.99288177490234 %
1
Catégorie prédite : 3
Véritable catégorie : 3
Probabilité de la prédiction : 99.99996185302734 %
2
Catégorie prédite : 3
Véritable catégorie : 3
Probabilité de la prédiction : 100.0 %
3
Catégorie prédite : 4
Véritable catégorie : 4
Probabilité de la prédiction : 99.99993896484375 %
4
Catégorie prédite : 4
Véritable catégorie : 4
Probabilité de la prédiction : 99.99603271484375 %
5
Catégorie prédite : 4
Véritable catégorie : 4
Probabilité de la prédiction : 100.0 %
6
Catégorie prédite : 0
Véritable catégorie : 0
Probabilité de la prédiction : 100.0 %
7
Catégorie prédite : 3
Véritable catégorie : 3
Probabilité de la prédiction : 99.9999771118164 %
8
Catégorie prédite : 2
Véritable catégorie : 2
Probabilité de la prédiction : 99.81684875488281 %
9
Catégorie prédite : 4
Véritable catégorie : 4
Probabilité de la prédiction : 100.0 %
10
Catégorie prédite : 4
Véritable catégorie : 4
Probabilit

In [107]:
filename = 'C:\Users\nicol\Desktop\NLPfinalized_model_Sentiment_analysis'
model.save(filename)

INFO:tensorflow:Assets written to: C:\Users\nicol\Desktop\NLPfinalized_model_Sentiment_analysis\assets
